In [2]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)
print(TORCH, CUDA)

2.0.1 cu118


In [3]:
!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 47.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 72.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.9/884.9 kB 23.7 M

In [4]:
import random
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/data/CiteSeer', name='CiteSeer')
print(len(dataset))
data = dataset[0]
data = data.to(device)
print(data)
num_node_features, num_classes = dataset.num_node_features, dataset.num_classes
print(num_node_features, num_classes)

Processing...
Done!


1
Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])
3703 6


In [6]:
import torch.nn.functional as F
import torch_geometric.nn as gnn
#gene: conv_type1, hidden_size, dropout1, conv_type2, dropout2
#conv_type: GCN, GIN, GAT, Cheb, Sage, K-GNN, fc 
hidden_choices = [8, 16, 32, 64, 128]
dropout_choices = [0.01, 0.25, 0.5, 0.75]

def generate_gene():
    ans = {}
    ans["conv_type1"] = np.random.randint(0, 7)
    ans["hidden_size"] = random.choice(hidden_choices)
    ans["dropout1"] = random.choice(dropout_choices)
    ans["conv_type2"] = np.random.randint(0, 7)
    ans["dropout2"] = random.choice(dropout_choices)
    return ans

def generate_population(num):
    return [generate_gene() for i in range(num)]


In [7]:
def layerchoice(layernum, in_channel, out_channel):
    if layernum == 1:
        return gnn.GINConv(nn=nn.Linear(in_channel, out_channel), eps=1e-9)
    elif layernum == 2:
        return gnn.GATConv(in_channel, out_channel)
    elif layernum == 3:
        return gnn.ChebConv(in_channel, out_channel, K=2)
    elif layernum == 4:
        return gnn.SAGEConv(in_channel, out_channel)
    elif layernum == 5:
        return gnn.GraphConv(in_channel, out_channel)
    elif layernum == 6:
        '''conv = gnn.GENConv(in_channel, out_channel, aggr='softmax',
                           t=1.0, learn_t=True, num_layers=2, norm='layer')
        act = nn.ReLU(inplace=True)
        return gnn.DeepGCNLayer(conv=conv, act=act, block='res+')'''
        return gnn.TransformerConv(in_channel, out_channel)
    else: #layernum == 0 or default
        return gnn.GCNConv(in_channel, out_channel)

class MyGNN(torch.nn.Module):
    def __init__(self, in_channels, out_channels, gene):
        super(MyGNN, self).__init__()
        hidden_size = 16 #default
        if "hidden_size" in gene:
            hidden_size = gene["hidden_size"]
        else:
            print("MyGNN init: hidden_size not in gene, choose 16 as default")

        self.conv1 = None
        if "conv_type1" not in gene:
            print("MyGNN init: conv_type1 not in gene, choose GCNConv as default")
            self.conv1 = gnn.GCNConv(in_channels, hidden_size)
        else:
            self.conv1 = layerchoice(gene["conv_type1"], in_channels, hidden_size)
        self.dropout1 = None
        if "dropout1" in gene:
            self.dropout1 = nn.Dropout(p=gene["dropout1"])
        else:
            print("MyGNN init: dropout1 not in gene, choose p=0.01 as default")
            self.dropout1 = nn.Dropout(p=0.01)

        self.conv2 = None
        if "conv_type2" not in gene:
            print("MyGNN init: conv_type2 not in gene, choose GCNConv as default")
            self.conv2 = gnn.GCNConv(hidden_size, out_channels)
        else:
            self.conv2 = layerchoice(gene["conv_type2"], hidden_size, out_channels)
        self.dropout2 = None
        if "dropout2" in gene:
            self.dropout2 = nn.Dropout(p=gene["dropout2"])
        else:
            print("MyGNN init: dropout2 not in gene, choose p=0.01 as default")
            self.dropout2 = nn.Dropout(p=0.01)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout2(x)
        x = F.softmax(x, dim=1)

        return x

In [8]:
def eval_gene(gene, epochs=200, myprint=False):
    print(gene)
    model = MyGNN(num_node_features, num_classes, gene).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    loss_function = torch.nn.CrossEntropyLoss().to(device)
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = loss_function(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        '''if myprint and epoch % 50 == 0:
            print("epoch {}, loss = {}".format(epoch, loss))'''
    model.eval()
    pred = model(data).argmax(dim=1)
    correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()
    acc = int(correct) / int(data.val_mask.sum())
    if myprint:
        print(f'Accuracy: {acc:.4f}')
    return acc

In [9]:
def genetic(population=8, iters = 20, mutation_rate=0.05):
    genes = generate_population(population)
    bestscore = []
    for iter in range(iters):
        scores = [eval_gene(g) for g in genes]
        print(scores)
        bestscore.append(max(scores))
        #choose highest half scores
        survive = np.argsort(-np.array(scores))[:population//2]
        genes = [genes[i] for i in survive]
        if (iter+1) % 1 == 0 or iter == iters-1:
            print("iteration=", str(iter+1), ", best parameters is", genes[0])
            print("accuracy is {:.2%}".format(bestscore[-1]))
        if iter == iters-1:
            break
        random.shuffle(genes)
        for i in range(len(genes)//2):
            g1, g2 = genes[i].copy(), genes[i+1].copy()
            #两两交配，完全打乱
            if np.random.randint(0,2):
                g1["conv_type1"], g2["conv_type1"] = g2["conv_type1"], g1["conv_type1"]
            if np.random.randint(0,2):
                g1["hidden_size"], g2["hidden_size"] = g2["hidden_size"], g1["hidden_size"]
            if np.random.randint(0,2):
                g1["dropout1"], g2["dropout1"] = g2["dropout1"], g1["dropout1"]
            if np.random.randint(0,2):
                g1["conv_type2"], g2["conv_type2"] = g2["conv_type2"], g1["conv_type2"]
            if np.random.randint(0,2):
                g1["dropout2"], g2["dropout2"] = g2["dropout2"], g1["dropout2"]
            genes.append(g1)
            genes.append(g2)
        while len(genes) < population:
            #e.g. population = 9 and after an iteration, population is 8
            genes.append(generate_gene())
        #mutation
        for i in range(population):
            if random.random() < mutation_rate:
                genes[i]["conv_type1"] = np.random.randint(0, 7)
            if random.random() < mutation_rate:
                genes[i]["hidden_size"] = random.choice(hidden_choices)
            if random.random() < mutation_rate:
                genes[i]["dropout1"] = max(0.01, min(0.99, genes[i]["dropout1"] + np.random.normal(scale=0.1)))
            if random.random() < mutation_rate:
                genes[i]["conv_type2"] = np.random.randint(0, 7)
            if random.random() < mutation_rate:
                genes[i]["dropout2"] = max(0.01, min(0.99, genes[i]["dropout2"] + np.random.normal(scale=0.1)))

    print("algorithm finished")
    plt.plot(list(range(1, iters+1)), bestscore)
    plt.title("score of best parameters")
    plt.show()

In [10]:
genetic()

{'conv_type1': 3, 'hidden_size': 128, 'dropout1': 0.01, 'conv_type2': 0, 'dropout2': 0.01}
{'conv_type1': 2, 'hidden_size': 128, 'dropout1': 0.25, 'conv_type2': 1, 'dropout2': 0.25}
{'conv_type1': 3, 'hidden_size': 128, 'dropout1': 0.5, 'conv_type2': 2, 'dropout2': 0.5}
{'conv_type1': 6, 'hidden_size': 64, 'dropout1': 0.25, 'conv_type2': 4, 'dropout2': 0.75}
{'conv_type1': 0, 'hidden_size': 8, 'dropout1': 0.5, 'conv_type2': 4, 'dropout2': 0.01}
{'conv_type1': 1, 'hidden_size': 16, 'dropout1': 0.01, 'conv_type2': 4, 'dropout2': 0.01}
{'conv_type1': 3, 'hidden_size': 32, 'dropout1': 0.25, 'conv_type2': 3, 'dropout2': 0.25}
{'conv_type1': 1, 'hidden_size': 16, 'dropout1': 0.5, 'conv_type2': 6, 'dropout2': 0.5}
[0.688, 0.618, 0.678, 0.444, 0.678, 0.652, 0.688, 0.636]
iteration= 1 , best parameters is {'conv_type1': 3, 'hidden_size': 128, 'dropout1': 0.01, 'conv_type2': 0, 'dropout2': 0.01}
accuracy is 68.80%
{'conv_type1': 0, 'hidden_size': 8, 'dropout1': 0.5, 'conv_type2': 4, 'dropout2': 

KeyboardInterrupt: ignored